# The Effects of Covid on the Stock Market
#### Analysis by: Don'yelle Allorh-quarcoo, David Ma, Dejan Savic, Mary Stewart, Wendell K
<p>As we are starting to recover from the recent pandemic of Covid-19, there is a ton of history to look back at in just a couple of years. Many topics might be coming to mind such as the shutdown, the explosion of TikTok, facemasks, working from home, and finding new hobbies to learn or do during the very, very gruesome couple months we had to spend alone or with family. One of those hobbies happened to be what our group likes to call, "Stonks!" For our very first project, we were interested to see how much of an effect Covid-19 had on the stock market before, during, and after.
    We decided to use an API module provided by Finnhub and data about Covid-19 provided by the World Health Organization and CDC to truly dive deep and code our way into the rabbit hole of "Stonks!" But, first we had to pull our data.</p>

**The Code**
<p>Let's get our hands dirty with Finnhub's API module. First, we imported a .csv file with a list of all tickers in the NASDAQ and NYSE exchanges. Then, we used a for loop with our API response to loop through all of the tickers and append Ticker, Company Name, Industry, Country, and Market Cap information to empty lists. From the DataFrame created by the empty lists, we filtered for all US companies with a market cap larger than $10B (Large-Cap). After dropping all duplicate tickers and N/A industries, we randomly sampled each industry for two stocks and created a DataFrame that we were then able to pull in historical stock price data from 2019-2021.</p>
<p>To make things go by even faster for data gathering, we had created functions to call multiple commands with multiple for loops. Instead of brute force coding and assigning a variable to each dataframe, we created a single dictionary for each year, then simply assigned the dataframe as the value to a key, acting as the "variable." We did the same for plotting as well to really speed through and be as efficient as possible in our code.</p> 
<p>You can take a look at the code below!

In [ ]:
# Loop through each industry to pick out all industries with more than 1 stock in it from our API response
two_stock_df = pd.DataFrame()
for industry in list(clean_company_industry.Industry.unique()):
    df_temp = clean_company_industry.loc[clean_company_industry['Industry']==industry]
    if len(df_temp) > 1:
        random_sample = random.sample(list(df_temp['Ticker']),k=2)
        df_temp = df_temp.loc[df_temp['Ticker'].isin(random_sample)]
        two_stock_df = two_stock_df.append(df_temp)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
two_stock_df

# Loop through every industry we are interested in and append the two stocks from each industry to a DataFrame
ten_industry_df = pd.DataFrame()
ten_industry_list = ['Airlines', 'Automobiles', 'Consumer products', 'Health Care', 'Hotels, Restaurants & Leisure', 'Logistics & Transportation', 'Pharmaceuticals', 'Real Estate', 'Telecommunication', 'Technology']
for industry in ten_industry_list:
    ten_industry_df = ten_industry_df.append(two_stock_df.loc[two_stock_df['Industry']==industry])
        
# Save the DataFrame to a .csv file
ten_industry_df.to_csv('output/ten_industry_df.csv', index=False)

In [2]:
# Define a function to retrieve a ticker's price for the year and format the columns
def stock_df(stock_name, beg_year, end_year):
    df_name = pd.DataFrame(finnhub_client.stock_candles(stock_name, 'W', beg_year, end_year)) 
    df_name = df_name.drop(['s'], axis=1)
    rename = {'c':'Close','h': 'High', 'l': 'Low', 'o': 'Open', 't': 'Time', 'v': 'Volume'}
    df_name.rename(columns=rename, inplace=True)
    df_name['Time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') for x in df_name['Time']]
    return df_name
    
# Define a function to create a stock chart for the price/date, volume, and cumulative Covid cases
def plot_chart_covid(dataframe, stock_name, year):
    fig= plt.figure(figsize=(12,10))
    top_plt = plt.subplot2grid((7,4), (0,0), rowspan=3, colspan=4)
    top_plt.plot(dataframe.index, dataframe['Close'], color = '#74D3AE')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Stock Price')
    plt.title(f'{stock_name} Performance Over the Year {year}')
    middle_plt = plt.subplot2grid((7,4),(3,0), rowspan=2, colspan=4)
    middle_plt.bar(dataframe.index, dataframe['Cumulative_cases'], color='#DD9787')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Cumulative Cases by Millions')
    bottom_plt = plt.subplot2grid((7,4),(5,0), rowspan=2, colspan=4)
    bottom_plt.bar(dataframe.index, dataframe['Volume'], color='#8A89C0')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Trading Volume by Millions')
    # plt.tight_layout();
    plt.close(fig) #added after saving/checking figs
    
# Define a function to create a stock chart for the price/date and volume
def plot_chart(dataframe, stock_name, year):
    fig= plt.figure(figsize=(12,10))
    top_plt = plt.subplot2grid((5,4), (0,0), rowspan=3, colspan=4)
    top_plt.plot(dataframe.index, dataframe['Close'],color='#74D3AE')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Stock Price')
    plt.title(f'{stock_name} Performance Over the Year {year}')
    bottom_plt = plt.subplot2grid((5,4),(3,0), rowspan=1, colspan=4)
    bottom_plt.bar(dataframe.index, dataframe['Volume'],color='#8A89C0')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Trading Volume by Millions')
    # plt.tight_layout();
    plt.close(fig) #added after saving/checking figs

**Pre-Covid, During Covid, Post-Covid**</p>
**Individual Stock Performance**
<p>We first took a look at each stocks performance over the years of Covid-19. The selected stocks that are depicted in the graph images display various performances throughout the year 2019-2021. One can see that each figure reflects that stocks were consistently volatile in 2020. Most of the stocks like AAPL, ALGN, CLX, CTLT, MSFT, and EL, performed better by the end of the year 2020 after the large initial drawdown in March 2020. For example, from the beginning of the year to the start of states implementing shutdowns around March 15, AAPL had a drawdown of -22.75%. Ultimately, it ended finishing the year up 79.98% for 2020. DAL, LVS, VZ, and VTRS are examples of stocks that appear to have a lower stock price at the end of 2020 than at the start of the year. For example, DAL had an initial drawdown of -63.84% from the beginning of the year to the start of state shutdowns around March 15. After the initial drawdown, prices recouped some of the losses, but still ended the year -31.89%.</p>

**Volume**
<p>There are many factors that come into play when we look at each stock's trading volume. For example, some of the factors that may influence trading volume may be earnings reports, geopolitical factors, litigation, periods of uncertainty, etc. It is shown that during Covid-19, the weekly volume of each stock varied but there was an overall trend in the volume of each stock. The period in which the WHO declared Covid-19 as a pandemic, states began implementing shutdowns, unemployment hit record highs, and cases started getting reported in most states (late February to early June) saw increased levels of trading volume and volatility for nearly all tickers that we analysed. For example, the Average Daily Trading Volume (ADTV) for AAPL in 2020 was 157,621,403 shares traded per day while the ADTV for 2019 and 2021 were 112,242,893 and 90,540,527, respectively. This can be attributed to the large spike in global market volume as the world was encountering a period of doubt and uncertainty.

**Covid Cases VS Stock Price**
<p>Our group had asked, "Could covid cases impact the prices of stonks?" Indeed it did. The number of covid cases had some sort of impact on the stock market and specific stocks during the year 2020. Luckily for those in the Consumer Product industry, as Covid cases started to rise, the price of various Consumer Products stocks rose as well. For example, CLX during covid (2020), actually benefited from the increase of Covid cases, posting a 34.68% increase for the year. Considering companies that sell staple items generally tend to perform better during times of uncertainty, this increase in CLX's price should be warranted. UPS is another good example in which covid cases and other underlying covid factors may have affected a stock's price positively. For 2020, UPS posted a 48.98% gain, which could be attributed to the rise in demand for the Logistics and Transportation industry after covid cases were on the rise and stay at home orders were being implemented. Apparently, everybody started needing to get their packages shipped to their homes once they couldn't go anywhere. Go figure!</p>
 <p>On the other hand, we have some stocks that were negatively affected by the increase in covid cases and other underlying covid factors. For example, UAL posted a whopping -51.81% loss for 2020. The Airlines industry stocks we looked at both posted similar returns and were among the select few that didn't fully recover the initial drawdown in March 2020. It is the obvious that customers will not be flying often, trapped in an air tight bacteria mobile, and their stock's return shows it.</p>

**Covid Milestones**
<p>Looking back at all the Covid-19 milestones shown in the covid_milestones.png, we can see that when the CDC announced "disruptions to everyday life may be severe" due to expected mitigation efforts and the WHO officially announced Covid-19 a pandemic (late February to mid March), a large majority of stocks had a dramatic drop in price and increase in volume. Following the announcement, states issuing shutdowns also impacted the price and volume of stocks during the year of 2020. The Airlines industry suffered the biggest drop and has yet to recover up to 2019 prices. For example, UAL closed trading for 2019 at a price of $89.09 per share. By the end of 2020, UAL closed at $43.25 per share, posting a -51.81% loss. Even to this day at the time of writing, UAL's stock price has never recovered back to 2019 prices, currently trading at $38.60 per share. Although, the Trump Administration's second Covid-19 relief bill and emergeceny use authorizations for vaccines (early December to early January 2021), did provide some much needed relief in overall stock prices.

**Linear Regression, Future Predictions, and Correlation Between Covid and Stocks**
<p>Although using a linear regression is a great way to create predictions, it doesn't help too much when we're talking about the stock market. Considering the stock market is affected by so many underlying factors, it is too hard to tell if the linear regression outputs are corresponsive of overlying market conditions or unique to each stock, or a mixture of both. If we take a look at each sectors performance, they all differed. For instance, the Airlines sector displayed a negative trend of stock prices during Covid-19 (2020) while the Technology and Healthcare industries showed positive trends. There seems to be somewhat of a correlation between stock price performance and Covid cases depending on each of the stocks we researched. For example only a handful of stocks (CLX, EL, MCK, LVS, PSA, AAPL, and MSFT) had a correlation coefficient greater than 0.70 with a pvalue less than 0.05. If we were to try and extrapolate the "future" price of one of these stock's, AAPL for example, we would multiply the slope of the linear regression by the amount of weeks out from the start of the regression and add that to the intercept. "Predicting" June 1, 2022 close price would estimate a close of about $185.48 when in fact the close price of AAPL on June 1, 2022 was $148.50. This indicates that using linear regression to predict future stock prices may not be the smartest move for an investor. Stocks don't always go up, sometimes they go down too!

**Stock Sector Performance**</p>
<p>After looking at the performance of individual stocks and realizing that some stocks move in the same direction while others don't, we decided to look at the bigger picture and investigate sector performance.</p>
<p>Let's take a step back and take a look at the sector performance in each of the sectors we were interested in.</p>
<p>We were be able to investigate how each sector was affected and how well each sector recovered or did not recover by using an average return for a bundle of stocks in a single sector for each sector. A random sample of stocks was chosen from the each of the following industries:</p>

In [ ]:
['Airlines', 'Automobiles', 'Consumer products', 'Health Care', 'Hotels, Restaurants & Leisure', 'Logistics & Transportation', 'Pharmaceuticals', 'Real Estate', 'Telecommunication', 'Technology']

<p>Using the random sample, the average increase/decrease was calculated for each industry and each year. Of note, TSLA was chosen in our random sample for the Automobile industry, but they had a stock split in 2020 which affected the API's underlying data as noted in the documentation. The average increase of 276.07% for the Automobile industry in 2020 is highly skewed by TSLA's increase of 684.16%. The Automobile industry average performance for 2020 should be approached with caution.</p>
<p>Overall, it can be seen that different sectors did in fact have a different performance over the course of the three years in question. Some had highly variable returns for each year, while some had more consistent returns for each year. Some had terrible starts to 2020, yet recovered fully by the end of the year while some never fully recovered **cough cough** Airlines...

**Biggest Sector Loser**
<p>A good handful of sectors did end up suffering during the pandemic (but also recovered), while the biggest loss was definitely in the Airlines industry. The Airlines industry's average loss for 2020 was -34.29%, while the second smallest return was 9.24% for the Real Estate industry. The 3 year average return for the Airlines industry was the only one still at a negative value (-8.38%) compared to the sectors in question in our analysis. If we take a closer look at the two stocks, DAL and UAL, they dipped just as hard as the other stocks in other sectors but weren't able to recover as quickly. Some other stocks in other industries that couldn't recover as quickly are VZ and FDX where they had a much later correction in stock prices, but were still able to out perform the Airlines industry in the end.</p>

**Biggest Sector Winners**
<p>The Logistics and Transportation sector performed the best during Covid-19 (2020) with an average increase of 44.95%, not including the skewed percentage for the Automobile industry. Although, the 3 year average return for the Logistics and Transportation industry was only 18.85% placing it sixth on our list of ten industries. These returns can be visualized by taking a look at the FDX and UPS stock price graphs, as they relatively track close together. The price of the stocks for UPS and FDX actively increases during the second half of 2020.</p>
<p>On a 3 year average return basis, the Technology sector (42.44%) takes the cake with the most sustained gain of all ten industries in question, with the Pharmaceuticals sector (33.27%) following close. The Tech industry posted an average return of 40.31% for 2020, placing it second on the list in 2020 returns, while the Pharmaceuticals industry placed only sixth for 2020. This makes sense after looking at the covid milestones section and realizing that vaccines did not really start being mass produced until late 2020. This could potentially explain why the Pharmaceutical industry had the best average return for 2021 (47.33%), with Real Estate following close behind at 44.87%.